In [24]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime

In [25]:
first_index = 0


In [26]:
file_path = 'C:\\Users\henry\Downloads\dota2.txt'
# Open the file in binary mode
with open(file_path, "rb") as file:
    # Read the binary content of the file
    binary_content = file.read()
decoded_content = binary_content.decode("utf-8", errors="replace")

soup = BeautifulSoup(decoded_content, 'html.parser')


In [27]:
# Create Dataframe if df does not already exist
columns = ['MatchID', 'Date', 'Hero', 'Enemy_pos1', 'Enemy_pos2', 'Enemy_pos3', 'Enemy_pos4', 'Enemy_po5',
           'Match Duration', 'Result', 'Kills', 'Deaths' 'Assists', 'Networth']

if 'df' not in locals():
    df = pd.DataFrame(columns)
    first_index = 0
else:
    first_index = first_index + 50


In [ ]:
for i in range(0,5):
    df['Enemy_pos' + str(i)] = ""

df['Networth'] = ""
df['Date'] = 

In [28]:
print(first_index)

0


In [29]:
print(df.head())

            0
0     MatchID
1        Date
2        Hero
3  Enemy_pos1
4  Enemy_pos2


In [30]:
heroes = soup.find_all("a", href=lambda href: href and '/matches/72' in href)
# Take info of myhero (Hero I played), matchid, and match result from html
hero_tags = soup.find_all('a', href=lambda href: href and '/matches/' in href)

matchids = []
heroes = []
results = []


for tag in hero_tags:

    # Extract hero name if the tag has "class" as an attribute
    if not tag.has_attr("class"):
        heroes.append(tag.text)

        # Extract the contents under the "href" attribute then split by "/" then take the MatchID
        matchid = tag['href'].split('/')[2]
        matchids.append(matchid)

     # Tags that don't have "class" attribute include the result of the match
    else:
        results.append(tag.text)


add_data = {'Hero' : heroes, 'MatchID' : matchids, 'Result' : results}
df_new_data = pd.DataFrame(add_data)

In [31]:
print(matchids)

['7298711135', '7280229073', '7268965457', '7266856262', '7255758726', '7255725098', '7255673780', '7250070281', '7247689570', '7247671586', '7247452281', '7246372234', '7239135242', '7239056825', '7238973938', '7238908864', '7238202411', '7238171114', '7238145508', '7238118885', '7238092792', '7237960065', '7237895477', '7232610806', '7231024516', '7228543993', '7227384223', '7227325085', '7227303780', '7227251683', '7227218819', '7227194113', '7227166001', '7227128373', '7227088056', '7227044961', '7227003847', '7226078987', '7226052094', '7226010839', '7225983519', '7221245229', '7218834860', '7217551342', '7216278139', '7216256731', '7216236208', '7216180843', '7216105413', '7216075145']


In [32]:
if df.shape[1] == 1:
    for i in range(0, 50):
        df.loc[i, 'Hero'] = heroes[i]
        df.loc[i, 'Result'] = results[i]
        df.loc[i, 'MatchID'] = matchids[i]
else:
    df = pd.concat([df, df_new_data], ignore_index = True)
print(df.head())

            0          Hero      Result     MatchID
0     MatchID        Oracle   Won Match  7298711135
1        Date  Witch Doctor   Won Match  7280229073
2        Hero          Lion   Won Match  7268965457
3  Enemy_pos1        Rubick  Lost Match  7266856262
4  Enemy_pos2        Rubick   Won Match  7255758726


In [33]:
for i in range(first_index, first_index + 50):
   current_match_id = df.loc[i,"MatchID"]
   starting_a_tag = soup.find_all('a', href=lambda href: href and '/matches/' + current_match_id in href)[1]  # Starting <a> tag for each matchid
   matchid_row_index = df[df['MatchID'] == current_match_id].index  # Find the row index of the match in the dataframe
   
   # Find the Date of the match
   date_tag_content = starting_a_tag.find_next('time')['title']
   date_time_format = datetime.strptime(date_tag_content, "%a, %d %b %Y %H:%M:%S %z")
   date = datetime.strftime(date_time_format, "%Y-%m-%d")
   df.loc[matchid_row_index, "Date"] = date

   # Find the duration of each match
   duration = starting_a_tag.find_all_next('td')[1].text  # Find the duration by finding the second <td> tag
   df.loc[matchid_row_index, 'Duration'] = duration  # Insert duration into row
   
   # Find the KDA (Kills/Deaths/Assists) of each match
   kda = starting_a_tag.find_all_next('td')[2].text  # Find the Kills/Deaths/Assists of the match
   df.loc[matchid_row_index, 'Kills'] = kda.split('/')[0]
   df.loc[matchid_row_index, 'Deaths'] = kda.split('/')[1]
   df.loc[matchid_row_index, 'Assists'] = kda.split('/')[2]

In [34]:
csv_dota2 = 'dota.csv'
df.to_csv(csv_dota2, index=False)


In [35]:
print(df.head())


            0          Hero      Result     MatchID        Date Duration  \
0     MatchID        Oracle   Won Match  7298711135  2023-08-23    50:02   
1        Date  Witch Doctor   Won Match  7280229073  2023-08-11    42:53   
2        Hero          Lion   Won Match  7268965457  2023-08-03    37:55   
3  Enemy_pos1        Rubick  Lost Match  7266856262  2023-08-02    47:16   
4  Enemy_pos2        Rubick   Won Match  7255758726  2023-07-25    25:18   

  Kills Deaths Assists  
0    10     10      20  
1    11     10      16  
2     5      6      31  
3     8     10      25  
4     3      1      14  


In [37]:
for i in df.columns:
    print(i)

0
Hero
Result
MatchID
Date
Duration
Kills
Deaths
Assists
